![Egeria Logo](https://raw.githubusercontent.com/odpi/egeria/main/assets/img/ODPi_Egeria_Logo_color.png)

### Egeria Coco Lab

# Setting Up A Clinical Trial

## Introduction

Clinical trials are used to test that new treatments are both safe and effective.  They involve taking measurements from various patients both before and after they start the treatment.

The data comes in from a variety of hospitals.  It is personally sensitive to the patients, of importance to the business, and subject to regulatory control and so care is needed that:

* it has been collected correctly
* it is protected at all times
* the correct data sharing agreements are in place to provide legal cover, both for the hospitals and Coco Pharmaceuticals.

We look at this process from the perspective of

1. The **business (clinical research)** who manage the clinical trial.  Tessa Tube is Coco Pharmaceutical's Chief Research Officer. She commissions any new trial, sets out the requirements and objectives for the study.  Tanya Tidie is responsible for the administration of the clinical trial.  She organizaes the data sharing agreements with the hospitals and monitors the progress to ensure all of the evidence is in place to show to the regulators.
   
2. The **data engineer** who sets up the data pipelines. Peter Profile is responsible for selecting the Unity Catalog that will protect the data from the clinical trial, setting up a schema and volume for the weekly measurements data in that catalog and, as the hospitals complete their registration into the clinical trail, activate the landing area and onboarding pipeline for data fro mthe hospital.

3. The **data scientist** who analyses the resulting data.  Callie Quartile is the data scientist who has to locate, access and analyse the data.  She needs to be sure the data is of sufficient quality and she has the right permissions to perform the type of analysis she wants.
   
<figure style="margin-left: 0%; display:inline-block;">
<img src="https://raw.githubusercontent.com/odpi/egeria-docs/main/site/docs/practices/coco-pharmaceuticals/personas/tessa-tube.png">
<figcaption style="margin-left: 15%;"><strong>Tessa Tube</strong></figcaption>
</figure>
    
<figure style="margin-left: 7%; display:inline-block;">
<img src="https://raw.githubusercontent.com/odpi/egeria-docs/main/site/docs/practices/coco-pharmaceuticals/personas/tanya-tidie.png">
<figcaption style="margin-left: 15%;"><strong>Tanya Tidie</strong></figcaption>
</figure>

<figure style="margin-left: 7%; display:inline-block;">
<img src="https://raw.githubusercontent.com/odpi/egeria-docs/main/site/docs/practices/coco-pharmaceuticals/personas/peter-profile.png">
<figcaption style="margin-left: 15%;"><strong>Peter Profile</strong></figcaption>
</figure>

<figure style="margin-left: 7%; display:inline-block;">
<img src="https://raw.githubusercontent.com/odpi/egeria-docs/main/site/docs/practices/coco-pharmaceuticals/personas/callie-quartile.png">
<figcaption style="margin-left: 15%;"><strong>Callie Quartile</strong></figcaption>
</figure>


----
## Checking environment set up

Lets get started by initializing the python interface to Egeria - pyegeria.

----

In [1]:
# Initialize pyegeria

%run ../pyegeria/initialize-pyegeria.ipynb

from datetime import datetime


----

The code below creates a client to validate the set up.  It is necessary to have the unity catalog integration connectors actively cataloguing Unity Catalog and the specialized governance engines for Coco Pharmaceuticals up and running.  This is achieved by running the HTTP Client collection called `AddCocoMetadataToQuickStartServers.http` and run the notebook [Survey and Catalog Unity Catalog](../cataloguing-and-surveys/unity-catalog/survey-and-catalog-uc.ipynb).

----

In [2]:
user_id = "erinoverview"
egeria_client = EgeriaTech(view_server, url, user_id, user_pwd)
token = egeria_client.create_egeria_bearer_token()

----

This content pack contains the definitions of the Coco Pharmaceuticals organization.

----

In [3]:

egeria_client.add_archive_file("content-packs/CocoComboArchive.omarchive", None, "qs-metadata-store")

print("CocoComboArchive loaded!")


CocoComboArchive loaded!


----

The next archive loads the PostgreSQL templates used when deploying the pipelines for the clinical trials.

----

In [4]:

egeria_client.add_archive_file("content-packs/PostgresContentPack.omarchive", None, "qs-metadata-store")

print("PostgresContentPack loaded!")


PostgresContentPack loaded!


----

Check that these governance engines that support the clinical trial are running ... 

----

In [5]:
display_gov_eng_status(['AssetOnboarding','AssetQuality@CocoPharmaceuticals','ClinicalTrials@CocoPharmaceuticals','UnityCatalogGovernance','UnityCatalogSurvey'],
                       status_filter=["*"],
                       engine_host = 'qs-engine-host',  view_server = 'qs-view-server',
                       paging = True, jupyter = True,width = 170,sort = True)

                                                           Governance Engine Status @ Fri Jun 20 17:38:20 2025                                                            
╭────────────────────────────────────┬────────────────────────┬────────────────────────────────────────────┬───────────────┬─────────────────────────────────────────────╮
│ Gov Engine                         │ Gov Engine Type        │ Gov Engine Desc                            │ Engine Status │ Request Types                               │
├────────────────────────────────────┼────────────────────────┼────────────────────────────────────────────┼───────────────┼─────────────────────────────────────────────┤
│ AssetOnboarding                    │ GovernanceActionEngine │ Monitors, validates and enriches metadata  │ RUNNING       │                                             │
│                                    │                        │ relating to assets as they are catalogued. │               │  • catalog-software-

----

A PostgreSQL database is used by the data scientist as part of their analysis of the data from the clinical trial.

----

In [6]:
# Set up the details of the PostgreSQL Server - check localhost is correct wrt where the OMAG Server PLatform is runnng.
## Use "localhost" if the OMAG Server Platform is runing natively; use "host.docker.internal" if the OMAG Server Plaform is running in docker (default)

postgres_host_identifier="localhost"
#postgres_host_identifier="host.docker.internal"

postgres_port_number="5432"
#postgres_port_number="5442"
postgres_database_name="clinical_trials"

postgres_server_name="LocalPostgreSQL1"
postgres_secrets_collection_name = "PostgreSQL Server:LocalPostgreSQL1"
secrets_store_path_name = "loading-bay/secrets/integration.omsecrets"


----

Data is received into the Coco Data Lake.  This is protected by multiple deployments of Unity Catalog Server one at each site with different types of storage attached.

A Unity Catalog Server can contain multiple catalogs.  The code below assumes that you have a Unity Catalog Server running that is catalogued as `Unity Catalog 1` and there is a catalog within it called `clinical_trials`.  The **Unity Catalog Survey and Catalog Workbook** will set this up.  You can change the names if you like.

----

In [7]:
# Set up the details of the Unity Catalog Server - check localhost is correct wrt where the OMAG Server PLatform is runnng.
## Use "http://localhost:8087" if the OMAG Server Platform is runing natively; use "http://host.docker.internal:8087" if the OMAG Server Plaform is running in docker (default)

uc_server_url="http://localhost:8087"
#uc_server_url="http://host.docker.internal:8087"

# Catalog name used by all clinical trials - set up in AddCocoMetadataToQuickStartServers.http
catalog_name="clinical_trials"

# If all is well, a goud should be returned for the catalog, indicating that 
catalog_guid=egeria_client.get_element_guid_by_unique_name("Unity Catalog Catalog::" + uc_server_url + "::" + catalog_name)
print(catalog_guid)

851f6039-ccfb-4118-a744-371f52a99850


-----

The next set of variables allow this notebook to run in different environments.  If you are using the standard quickstart environment then there is no need to change these values.

-----

In [8]:

tessas_user_id = "tessatube"
tessas_view_server=view_server
tessas_view_server_url=url
tessas_user_pwd = os.environ.get("EGERIA_USER_PASSWORD")

tanyas_user_id = "tanyatidie"
tanyas_view_server=view_server
tanyas_view_server_url=url
tanyas_user_pwd = os.environ.get("EGERIA_USER_PASSWORD")

callies_user_id = "calliequartile"
callies_view_server=view_server
callies_view_server_url=url
callies_user_pwd = os.environ.get("EGERIA_USER_PASSWORD")

peters_user_id="peterprofile"
peters_view_server=view_server
peters_view_server_url=url
peters_user_pwd = os.environ.get("EGERIA_USER_PASSWORD")



----

## Clinical Trial Solution Blueprint

This query shows the solution blueprint for all clinical trial processing at Coco Pharmaceuticals.  

Solution blueprint definitions are created through a simple markdown document that is loaded into Egeria using the Dr.Egeria feature.  It is a generic description of how a particular type of project (clinical trials in this example) operate.  A solution blueprint contains a description of the people (or more specifically the roles of the people) involved in this type of project along with both the manual and automated processes in use. It provides a convenient structure for linking descriptions of the governance requirements and responsibilities into a business operational view.  Thus the solution blueprint consolidates the aspects of an activity that the organisation wishes to be consistent.  This may be for cost, or quality and/or regulatory reasons.
The solution blueprint also provides a familiar structure to to report the operational status of each part of the end-to-end processing

Now look at this solution blueprint in detail.  

If we start from the top, there is data coming in from one or more hospitals, and each hospital has a coordinator that ensures the hospital is delivering the data as agreed.  We don’t really know exactly how each hospital manages their data - just that whatever processes they have, will deliver the appropriate data into the landing area folder for the hospital.  

The delivery of data from a hospital is immediately detected by the landing folder cataloguer which initiates an onboarding request for the data.  

The onboarding pipeline is responsible for moving the data to the safety of the data lake and ensuring it is correctly catalogued.  This is a critical step, and it is set up and managed by a data engineer, who is supported by the clinical trial manager if there are issues with the data from any of the hospitals.  Once  the data is in the data lake, it is picked up by the clinical trials researchers and data scientists for analysis and ultimately to deliver a report summarising the findings of the clinical trial.

Notice that down the centre of the solution blueprint is a high level view of how the data for the clinical trial flows.   The processes and roles that support this data flow are linked to either side.


In [9]:

print_solution_blueprints(egeria_client, "Clinical Trial Management Solution Blueprint")


----

Since the solution blueprint is generic, projects are used to track the progress of individual clinical trials.  These are created using templates associated with the solution blueprint.  These templates help to ensure consistency from project to project.   One of the key templates is the information suppy chain template.  This identifies which components in the solution blueprint are responsible for lineage.  It also divides the data flow into segments.  A segment shows the part of the data flow that is the responsibility of a particular team, organization or persion.  For clinical trials, this includes:

* The delivery of data from the hospitals
* The onboarding and validating of data from the hospitals into the data lake
* The provisioning of data into the sandboxes used for analysing the clinical trial measurements
* The assessment of the clinical trial data
* The reporting of the results


Here is the information supply chain template for clinical trials ... notice that some solution components are linked to multiple solution components.  This signifies a handover of responsibility for the data.

----

In [10]:

print_information_supply_chains(egeria_client, "Clinical Trials Information Supply Chain", True, True)


In [11]:

print_information_supply_chains(egeria_client, "Clinical Trial::{{clinicalTrialId}}", True, True)


In [12]:

print_information_supply_chains(egeria_client, "Clinical Trial Treatment Validation::{{clinicalTrialId}}", True, True)


-----

Other [templates](https://egeria-project.org/features/templated-cataloguing/overview/) are used to ensure that as digital resources are catalogued, they have the right structure, classifications and attachments.

----

In [ ]:
# Identify the standard onboarding pipeline

onboarding_pipeline_name = "Onboard Landing Area Files For Clinical Trial Project"
onboarding_pipeline_guid = egeria_client.get_guid_for_name(onboarding_pipeline_name) 
print(onboarding_pipeline_guid)


# Identify the templates that are used to create consistency in the clinical trial process

data_lake_schema_template_guid=TEMPLATE_GUIDS['Unity Catalog Schema']
print("dataLakeSchemaTemplateGUID: " + data_lake_schema_template_guid)
print_asset_graph_by_guid(egeria_client, data_lake_schema_template_guid)

data_lake_volume_template_guid=TEMPLATE_GUIDS['Unity Catalog Volume']
print("dataLakeVolumeTemplateGUID: " + data_lake_volume_template_guid)
print_asset_graph_by_guid(egeria_client, data_lake_volume_template_guid)

data_lake_file_template_guid = egeria_client.get_element_guid_by_unique_name("DataLake::{{clinicalTrialId}}::WeeklyMeasurements::{{hospitalName}}::{{fileName}}")
print("dataLakeFileTemplateGUID: " + data_lake_file_template_guid)
print_asset_graph_by_guid(egeria_client, data_lake_file_template_guid)

landing_area_directory_template_guid = egeria_client.get_element_guid_by_unique_name("Data Folder::{{fileSystemName}}.{{directoryPathName}}")
print("landingAreaDirectoryTemplateGUID: " + landing_area_directory_template_guid)
print_asset_graph_by_guid(egeria_client, landing_area_directory_template_guid)

landing_area_file_template_guid = egeria_client.get_element_guid_by_unique_name("LandingArea::{{hospitalName}}::{{clinicalTrialId}}::WeeklyMeasurements::{{fileName}}")
print("landingAreaFileTemplateGUID: " + landing_area_file_template_guid)
print_asset_graph_by_guid(egeria_client, landing_area_file_template_guid)

validated_weekly_files_template_guid = egeria_client.get_element_guid_by_unique_name("Data File Collection::{{displayName}}")
print("validatedWeeklyFilesTemplateGUID: " + validated_weekly_files_template_guid)
print_asset_graph_by_guid(egeria_client, validated_weekly_files_template_guid)

---

At Coco Pharmaceuticals, clinical trial projects are part of the *Clinical Trial Management* campaign.  This is a project that is responsible for the generic resources used by all clinical trials.

---

In [ ]:

parent_project_name="Campaign:Clinical Trials Management"
parent_project_guid=egeria_client.get_element_guid_by_unique_name(parent_project_name)

print("Parent project: " + parent_project_name)
print("  parent project GUID: " + parent_project_guid)

print_project(egeria_client, parent_project_guid)


----

The definitions and functions that follow are generic for all clinical trials that are set up in this notebook.  They are added here to simplify the steps in each clinical trial.

----

In [ ]:
# These functions drive calls to Egeria at different phases of the clinical trial


def set_up_clinical_trial(egeria_tech,parent_project_guid, clinical_trial_owner_guid, clinical_trial_manager_guid, project_manager_guid, data_engineer_guid, integration_developer_guid, data_scientist_guid, project_identifier, project_name, project_description): 
    set_up_clinical_trial_name="ClinicalTrials@CocoPharmaceuticals::set-up-clinical-trial"
    action_targets = [{
          "class" : "NewActionTarget",
          "actionTargetName": "clinicalTrialParentProject",
          "actionTargetGUID": parent_project_guid
        },
        {
          "class" : "NewActionTarget",
          "actionTargetName": "clinicalTrialOwner",
          "actionTargetGUID": clinical_trial_owner_guid 
        },
        {
          "class" : "NewActionTarget",
          "actionTargetName": "clinicalTrialManager",
          "actionTargetGUID": clinical_trial_manager_guid 
        },
        {
          "class" : "NewActionTarget",
          "actionTargetName": "itProjectManager",
          "actionTargetGUID": project_manager_guid
        },
        {
          "class" : "NewActionTarget",
          "actionTargetName": "dataEngineer",
          "actionTargetGUID": data_engineer_guid
        },
        {
          "class" : "NewActionTarget",
          "actionTargetName": "integrationDeveloper",
          "actionTargetGUID": integration_developer_guid
        },
        {
          "class" : "NewActionTarget",
          "actionTargetName": "dataScientist",
          "actionTargetGUID": data_scientist_guid
        }]
    
    request_parameters = request_parameters = {
         "clinicalTrialId" : project_identifier,
         "clinicalTrialName" : project_name,
         "clinicalTrialDescription" : project_description
    }
    
    egeria_tech.initiate_gov_action_type(set_up_clinical_trial_name, None, action_targets, None, request_parameters, None, None)



def nominate_hospital(egeria_tech, project_identifier, hospital_guid, contact_person_guid):
    nominate_hospital_name = "ClinicalTrials::" + project_identifier + "::nominate-hospital"
    process_guid = egeria_tech.get_element_guid_by_unique_name(nominate_hospital_name)
    print(nominate_hospital_name + " [" + process_guid + "] running for hospital: " + hospital_guid)
    if process_guid != "No elements found":
        action_targets = [{
              "class" : "NewActionTarget",
              "actionTargetName": "hospital",
              "actionTargetGUID": hospital_guid
            },
            {
              "class" : "NewActionTarget",
              "actionTargetName": "hospitalContactPerson",
              "actionTargetGUID": contact_person_guid
            }]
        
        egeria_tech.initiate_gov_action_process(nominate_hospital_name, None, action_targets, datetime.now(), None, None, None)



def set_up_data_lake(egeria_tech, project_identifier, project_name, project_directory_name, project_schema_name):
    set_up_data_lake_process_name="ClinicalTrials::" + project_identifier + "::set-up-data-lake"
    process_guid = egeria_tech.get_element_guid_by_unique_name(set_up_data_lake_process_name)
    print(set_up_data_lake_process_name + " [" + process_guid + "] running")
    if process_guid != "No elements found":    
        data_lake_directory_path_name="coco-data-lake/research/clinical-trials/" + project_directory_name + "/weekly-measurements"
        
        airflow_dag_name="populate_" + project_schema_name + "_sandbox"
        
        action_targets = [{
              "class" : "NewActionTarget",
              "actionTargetName": "dataLakeCatalog",
              "actionTargetGUID": catalog_guid
            },
            {
              "class" : "NewActionTarget",
              "actionTargetName": "onboardingPipeline",
              "actionTargetGUID": onboarding_pipeline_guid 
            }]
        
        request_parameters = {
             "dataLakeVolumeTemplateGUID" : data_lake_volume_template_guid,
             "dataLakeSchemaTemplateGUID" : data_lake_schema_template_guid,
             "dataLakeFileTemplateGUID" : data_lake_file_template_guid,
             "landingAreaDirectoryTemplateGUID" : landing_area_directory_template_guid,
             "landingAreaFileTemplateGUID" : landing_area_file_template_guid,
             "dataLakeSchemaName" : project_schema_name,
             "dataLakeSchemaDescription" : "Data for the " + project_name + ".",
             "dataLakeVolumeName" : "weekly_measurements",
             "dataLakeVolumeDescription" : "Weekly patient measurements",
             "dataLakeVolumeDirectoryPathName" : data_lake_directory_path_name,
             "validatedWeeklyFilesDataSetName" : "Validated Incoming Weekly Measurements for the " + project_name + ".",
             "validatedWeeklyFilesTemplateGUID" : validated_weekly_files_template_guid,
             "airflowDAGName" : airflow_dag_name,
             "serverName" : postgres_server_name,
             "hostIdentifier" : postgres_host_identifier,
             "portNumber" : postgres_port_number,
             "secretsStorePathName" : secrets_store_path_name,
             "secretsCollectionName" : postgres_secrets_collection_name,
             "versionIdentifier" : "1.0",
             "databaseName" : postgres_database_name,
             "schemaName" : project_schema_name,
             "schemaDescription" : "PostgreSQL database schema for the " + project_name + "."
        }
        
        egeria_tech.initiate_gov_action_process(set_up_data_lake_process_name, None, action_targets, None, request_parameters, None, None)
        return airflow_dag_name



def certify_hospital(egeria_tech, project_identifier, hospital_guid):
    certify_hospital_name = "ClinicalTrials::" + project_identifier + "::certify-hospital"
    process_guid = egeria_tech.get_element_guid_by_unique_name(certify_hospital_name)
    print(certify_hospital_name + " [" + process_guid + "] running for hospital: " + hospital_guid)
    if process_guid != "No elements found":
        action_targets = [{
              "class" : "NewActionTarget",
              "actionTargetName": "hospital",
              "actionTargetGUID": hospital_guid
            }]
        egeria_tech.initiate_gov_action_process(certify_hospital_name, None, action_targets, datetime.now(), None, None, None)



def onboard_hospital(egeria_tech, project_identifier, project_directory_name, hospital_guid, hospital_landing_area_folder):
    onboard_hospital_name = "ClinicalTrials::" + project_identifier + "::onboard-hospital"
    process_guid = egeria_tech.get_element_guid_by_unique_name(onboard_hospital_name)
    print(onboard_hospital_name + " [" + process_guid + "] running for hospital: " + hospital_guid)
    if process_guid != "No elements found":
        landing_area_directory_name="landing-area/hospitals/" + hospital_landing_area_folder + "/clinical-trials/" + project_directory_name
        actionTargets = [{
              "class" : "NewActionTarget",
              "actionTargetName": "hospital",
              "actionTargetGUID": hospital_guid
            }]
        requestParameters = {
            "landingAreaDirectoryPathName" : landing_area_directory_name
        }
        egeria_tech.initiate_gov_action_process(onboard_hospital_name, None, actionTargets, datetime.now(), requestParameters, None, None)
        return landing_area_directory_name
    else:
        return None

In [ ]:
# These are the sample data files found in Egeria's distribution

oak_dene_source_folder   = 'loading-bay/sample-data/oak-dene-drop-foot-weekly-measurements'
old_market_source_folder = 'loading-bay/sample-data/old-market-drop-foot-weekly-measurements'
hampton_source_folder    = 'loading-bay/sample-data/hampton-drop-foot-weekly-measurements'

# This function simulates the delivery of a file from a hospital.

def add_file_to_landing_area(egeria_client, source_folder, destination_folder, week_number):
    add_file_to_landing_area_name="ClinicalTrials@CocoPharmaceuticals::simulate-ftp"
    source_file_name = source_folder + "/" + "week" + week_number + ".csv"
    print("Moving " + source_file_name + " to " + destination_folder)
    request_parameters = {
        "sourceFile" : source_file_name,
        "destinationDirectory" : destination_folder
    }
    egeria_client.initiate_gov_action_type(add_file_to_landing_area_name, None, None, None, request_parameters, None)
    

----

## Teddy Bear Drop Foot Clinical Trial

In this first clinical trial, we step through each phase slowly and observe the behaviour of the associated templates and governance actions.  

### Initializing the clinical trial

Tessa Tube is the lead researcher at Coco Pharmaceuticals.  She is the *Clinical Trial Sponsor*.   In addition to monitoring the clinical trial throughout its lifetime, she has two specific actions to perform, one at the start of the trial and one at the end:


---

In [ ]:

tessas_client = EgeriaTech(tessas_view_server, tessas_view_server_url, tessas_user_id, tessas_user_pwd)
token = tessas_client.create_egeria_bearer_token()

print_solution_roles(tessas_client, "Clinical Trial Sponsor")


----

The clinical trial that Tessa is setting up is for the `Teddy Bear Drop Foot` condition that afflicts elderly teddy bears as their stuffung sags, loosening their hip joints.  The clinical trial is given an identifier, name and description along with resource names used to set up specific file directories and schema for the data being processed by each stage in the clinicl trial.

----

In [ ]:
print()

project_identifier="PROJ-CT-TBDF"
project_name="Teddy Bear Drop Foot Clinical Trial"
project_description="Clinical trial related to the new treatment for Teddy Bear Drop Foot."
project_directory_name="teddy-bear-drop-foot"
project_schema_name="teddy_bear_drop_foot"

print("Project " + project_identifier + ": " + project_name)
print("  " + project_description)

print()


----

The people involved in the clinical trial are identified.  There is a person for each of the solution roles in the blueprint.

----

In [ ]:
print()

clinical_trial_owner_name="Tessa Tube"
print("clinical trial owner name: " + clinical_trial_owner_name)
clinical_trial_owner_guid = tessas_client.get_guid_for_name(clinical_trial_owner_name)
print("clinical trial owner GUID: " + clinical_trial_owner_guid)

print()
clinical_trial_manager_name="Tanya Tidie"
print("clinical trial manager name: " + clinical_trial_manager_name)
clinical_trial_manager_guid = tessas_client.get_guid_for_name(clinical_trial_manager_name)
print("clinical trial manager GUID: " + clinical_trial_manager_guid)

print()
data_scientist_name="Callie Quartile"
print("data scientist name: " + data_scientist_name)
data_scientist_guid = tessas_client.get_guid_for_name(data_scientist_name)
print("data scientist GUID: " + data_scientist_guid)

print()
data_engineer_name="Peter Profile"
print("data engineer name: " + data_engineer_name)
data_engineer_guid = tessas_client.get_guid_for_name(data_engineer_name) 
print("data engineer GUID: " + data_engineer_guid)

print()
project_manager_name="Polly Tasker"
print("project manager name: " + project_manager_name)
project_manager_guid = tessas_client.get_guid_for_name(project_manager_name) 
print("project manager GUID: " + project_manager_guid)

print()
integration_developer_name="Bob Nitter"
print("integration developer name: " + integration_developer_name)
integration_developer_guid = tessas_client.get_guid_for_name(integration_developer_name) 
print("integration developer GUID: " + integration_developer_guid)

print()

---

Once Tessa has an agreed scope and purpose for a new clinical trial, she is ready to set up the project resources that will control the clinical trial.

---

In [ ]:

print_solution_components(tessas_client, "Set up clinical trial")


---

The `set-up-clinical-trial` governance action type creates the specific governance processes that will be used by the staff during the clinical trial.  It uses generic process definitions to create these specific processes that are initialized with all of the correct values for the clinical trial.  This approach reduces the chance that someone will use a wrong value by accident when running the later processes for the clinical trial.

---

In [ ]:
print_anchor_graph(tessas_client, "ClinicalTrials@CocoPharmaceuticals::set-up-clinical-trial")

In [ ]:

set_up_clinical_trial(tessas_client,
                      parent_project_guid,
                      clinical_trial_owner_guid,
                      clinical_trial_manager_guid,
                      project_manager_guid,
                      data_engineer_guid,
                      integration_developer_guid,
                      data_scientist_guid,
                      project_identifier, 
                      project_name,
                      project_description) 


----

The command below shows the status of the governance action.  Run (and re-run) the cell below until you see the status "ACTIONED".  You may see statuses such as "PROPOSED", "APPROVED", "IN_PROGRESS" but not "FAILED".

----

In [ ]:

display_engine_activity_c(row_limit=1,width=170)


-----

The set-up-clinical-trial governance action has created the following processes that are specific to this clinical trial and are set up with details of the correct project and people.

-----

In [ ]:

print_matching_gov_action_processes(tessas_client, project_identifier)


---

These processes are linked to the appropriate tasks in the new clinical trial project that were also defined by the set up process...

---

In [ ]:

project_guid=tessas_client.get_element_guid_by_unique_name("Project::" + project_identifier + "::" + project_name)
print(project_guid)
if project_guid != "No elements found":
    print_project(tessas_client, project_guid)
    print_project_graph(tessas_client, project_guid)

print_project_graph(egeria_client, parent_project_guid)


----

Notice an empty data specification has been created and linked to the appropriate tasks.  The dark lines show the dependencies between  the tasks.  Completing this data specification is required before many tasks can proceed.

There is also the [information supply chain](https://egeria-project.org/concepts/information-supply-chain/) that identifies the lineage that is relevant to this project.  Notice that the project details are filled out in the placeholders of the information supply chain template.  On the left are the **segments** showing how the responsbility for the data is passed from team to team.  These are linked to the relevant solution components supporting the data processing/movement.  As the project progresses and resources are deployed to support it, these resources will appear in the implementation of the information supply chain.

---

In [ ]:

print_information_supply_chains(tessas_client, "Clinical Trial Treatment Validation::" + project_identifier, True, True)


----

### Nominating Hospitals

Tanya Tidie is the clinical trial manager, responsible for the business arrangements for each clinical trial. This includes managing the relationships with the hospitals, ensuring all of the "paperwork" associated with the clinical trial is in place for the sponsors and regulators as well as handling any issues with the data received from the hospitals.

----

In [ ]:

tanyas_client = EgeriaTech(tanyas_view_server, tanyas_view_server_url, tanyas_user_id, tanyas_user_pwd)
token = tanyas_client.create_egeria_bearer_token()

print_solution_roles(tanyas_client, "Clinical Trial Manager")


---

These are the hospitals and associated contact people that Tanya works with.

----

In [ ]:

# Identify the hospitals that take part in one or more clinical trials.

print()

oak_dene_hospital_name="Oak Dene Hospital"
print(oak_dene_hospital_name)
oak_dene_hospital_guid=tanyas_client.get_element_guid_by_unique_name("Organization::" + oak_dene_hospital_name)
print("  oakDeneHospitalGUID: " + oak_dene_hospital_guid)
oak_dene_contact_person_name="Robbie Records"
print("  oakDeneContactPersonName: " + oak_dene_contact_person_name)
oak_dene_contact_person_guid=tanyas_client.get_guid_for_name(oak_dene_contact_person_name)
print("  oakDeneContactPersonGUID: " + oak_dene_contact_person_guid)
oak_dene_landing_folder_directory="oak-dene"

print()
old_market_hospital_name="Old Market Hospital"
print(old_market_hospital_name)
old_market_hospital_guid=tanyas_client.get_element_guid_by_unique_name("Organization::" + old_market_hospital_name)
print("  oldMarketHospitalGUID: " + old_market_hospital_guid)
old_market_contact_person_name="Nellie Dunn"
print("  oldMarketContactPersonName: " + old_market_contact_person_name)
old_market_contact_person_guid=tanyas_client.get_guid_for_name(old_market_contact_person_name)
print("  oldMarketContactPersonGUID: " + old_market_contact_person_guid)
old_market_landing_folder_directory="old-market"


print()
hampton_hospital_name="Hampton Hospital"
print(hampton_hospital_name)
hampton_hospital_guid=tanyas_client.get_element_guid_by_unique_name("Organization::" + hampton_hospital_name)
print("  hamptonHospitalGUID: " + hampton_hospital_guid)
hampton_contact_person_name="Grant Able"
print("  hamptonContactPersonName: " + hampton_contact_person_name)
hampton_contact_person_guid=tanyas_client.get_guid_for_name(hampton_contact_person_name)
print("  hamptonContactPersonGUID: " + hampton_contact_person_guid)
hampton_landing_folder_directory="hampton"


print()
bowden_arrow_hospital_name="Bowden Arrow Hospital"
print(bowden_arrow_hospital_name)
bowden_arrow_hospital_guid=tanyas_client.get_element_guid_by_unique_name("Organization::" + bowden_arrow_hospital_name)
print("  bowdenArrowHospitalGUID: " + bowden_arrow_hospital_guid)
bowden_arrow_contact_person_name="Julie Stitched"
print("  bowdenArrowContactPersonName: " + bowden_arrow_contact_person_name)
bowden_arrow_contact_person_guid=tanyas_client.get_guid_for_name(bowden_arrow_contact_person_name)
print("  bowdenArrowContactPersonGUID: " + bowden_arrow_contact_person_guid)
bowden_arrow_landing_folder_directory="bowden-arrow"
print()



----

Tanya's work with the hospital is tracked under the "HospitalManagement" task for this project.  This task is dependent on completing the data specification for the project sicne it plays a major role when setting up the data sharing agreements with the hospitals.

----

In [ ]:

project_guid=tanyas_client.get_element_guid_by_unique_name("Project::" + project_identifier + "::HospitalManagement")
print(project_guid)

if project_guid != "No elements found":
    print_project_graph(tanyas_client, project_guid)


----

Once the data specificaiton is in place, and the basic data sharing agreement has been defined, Tanya begins to work with the hospitals that will take part in the clinical trial.  For each one, she runs nominate-hospital to identify the people at the hospital that are going to complete the requirements (training, signing data sharing agreements, ...) necessary to participate.

----

In [ ]:

print_solution_components(tanyas_client, "Nominate Hospital")


----

In this clinical trial, patients are associated with three hospitals:

* Oak Dene Hospital
* Old Market Hospital
* Hampton Hospital

----

In [ ]:

# Nominate each hospital

nominate_hospital(tanyas_client, project_identifier, oak_dene_hospital_guid, oak_dene_contact_person_guid)
nominate_hospital(tanyas_client, project_identifier, old_market_hospital_guid, old_market_contact_person_guid)
nominate_hospital(tanyas_client, project_identifier, hampton_hospital_guid, hampton_contact_person_guid)


In [ ]:

# Check that each nominate hospital process is "ACTIONED"

display_engine_activity_c(row_limit=3, width=150)

In [ ]:

print_related_elements_for_name(tanyas_client, 'CertificationType:: Participating Hospital for an approved Clinical Trial', "Certification")

mermaidGraph="---\ntitle: Certifications - Participating Hospital for an approved Clinical Trial [5acc69bf-dfbb-4e4a-b47b-19d610f2cb06]\n---\nflowchart LR\n%%{init: {\"flowchart\": {\"htmlLabels\": false}} }%%\n\n1@{ shape: doc, label: \"*Certification Type*\n**Participating Hospital for an approved Clinical Trial**\"}\n2@{ shape: text, label: \"*Title*\n**Participating Hospital for an approved Clinical Trial**\"}\n3@{ shape: text, label: \"*Summary*\n**The hospital is participating in the associated Clinical Trial.**\"}\n2~~~3\n4@{ shape: text, label: \"*Description*\n**This certification is started when the hospital completes the necessary preparations for the clinical trail.**\"}\n3~~~4\n5@{ shape: trap-t, label: \"*Organization*\n**Oak Dene Hospital**\"}\n1==>|\"Certification\"|5\n6@{ shape: trap-t, label: \"*Person*\n**Tessa Tube**\"}\n5-->|\"certifiedBy\"|6\n7@{ shape: trap-t, label: \"*Person*\n**Tanya Tidie**\"}\n5-->|\"custodian\"|7\n8@{ shape: trap-t, label: \"*Person*\n**Robbie Records**\"}\n5-->|\"recipient\"|8\n9@{ shape: trap-t, label: \"*Organization*\n**Hampton Hospital**\"}\n1==>|\"Certification\"|9\n9-->|\"certifiedBy\"|6\n9-->|\"custodian\"|7\n10@{ shape: trap-t, label: \"*Person*\n**Grant Able**\"}\n9-->|\"recipient\"|10\n11@{ shape: trap-t, label: \"*Organization*\n**Old Market Hospital**\"}\n1==>|\"Certification\"|11\n11-->|\"certifiedBy\"|6\n11-->|\"custodian\"|7\n12@{ shape: trap-t, label: \"*Person*\n**Nellie Dunn**\"}\n11-->|\"recipient\"|12\nstyle 11 color:#000000, fill:#FFE599, stroke:#000000\nstyle 1 color:#FFFFFF, fill:#006400, stroke:#000000\nstyle 12 color:#000000, fill:#FFE599, stroke:#000000\nstyle 2 color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle 3 color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle 4 color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle 5 color:#000000, fill:#FFE599, stroke:#000000\nstyle 6 color:#000000, fill:#FFE599, stroke:#000000\nstyle 7 color:#000000, fill:#FFE599, stroke:#000000\nstyle 8 color:#000000, fill:#FFE599, stroke:#000000\nstyle 9 color:#000000, fill:#FFE599, stroke:#000000\nstyle 10 color:#000000, fill:#FFE599, stroke:#000000\n"
render_mermaid(mermaidGraph)

----

### Setting up the data lake

Peter Profile is a data engineer at Coco Pharmaceuticals and is reponsible for the data pipelines and data stores that support the clinical trial.

---

In [ ]:

peters_client = EgeriaTech(peters_view_server, peters_view_server_url, peters_user_id, peters_user_pwd)
token = peters_client.create_egeria_bearer_token()

print_solution_roles(peters_client, "Certified Data Engineer")


-----

Peter first task is to set up the data and the data stores that are independent of the hospital supplying the data.  This includes components and templates that are influenced by the shape of the data needed by Coco Pharmaceuticals to validate their new treatment.  Like, Tanya, Peter can begin setting up the data lake and data stores once the data dpecification is agreed since it is used to generate the correct schemas for these stores.

The data specification is also used at a later stage when each hospital is onboarded since the data validator in the onboarding pipeline dynamically validates that the incoming data conforms to the requirements laid out in the data specification.

---

In [ ]:

project_guid=peters_client.get_element_guid_by_unique_name("Project::" + project_identifier + "::OnboardingPipelines")
print(project_guid)

if project_guid != "No elements found":
    print_project_graph(peters_client, project_guid)


----

However the first stage is to lay down the hospital independent data stores in the data lake and sandbox used by the data scientist and researchers.

----

In [ ]:

print_solution_components(peters_client, "Set up Data Lake")


----

The function below runs the set up data lake for this project.

----

In [ ]:

airflow_dag_name=set_up_data_lake(peters_client, project_identifier, project_name, project_directory_name, project_schema_name)

print(airflow_dag_name)

In [ ]:
## Check that set up data lake is ACTIONED

display_engine_activity_c(row_limit=1, width=170)


----

One of the services that is configured by "set up data lake" is the integration connector that monitors new files in the Unity Catalog volume that holds the validated files received from the hospitals.  It records the time that the last file was received to help the data scientists to determine if new data has arrived since they last ran their analysis.


In [ ]:

display_integration_daemon_status(['MaintainLastUpdateDate', 'FilesCataloguer'], paging=True, width=170)


----

It is now possible to see the part of the information supply chain that has just been set up.  It is shown in the "implementation" grey box.  Notice that each component is linked to the solution component it represents.

---

In [ ]:

print_information_supply_chains(peters_client, "Clinical Trial Treatment Validation::" + project_identifier, True, True)


-----

The first part of the information supply chain will be added when the hospitals are cenrtified and onboarded into the clinical trial.

-----

### Certifying Hospitals (Round One)

Once a hospital has completed all of the requirements, it can be certified for the clinical trial.  This means it can supply data to the clinical trial.  At the start of the clinical trial, Robbie Record, for Oak Dene and Grant Able for Hampton Hospital are ready to go.  Nellie Dunn still does not have the data sharing agreements signed for Old Market.  As a result, only Oak Dene and Hampton hospitals are certified.

----

In [ ]:

tanyas_client = EgeriaTech(tanyas_view_server, tanyas_view_server_url, tanyas_user_id, tanyas_user_pwd)
token = tanyas_client.create_egeria_bearer_token()

certify_hospital(tanyas_client, project_identifier, oak_dene_hospital_guid)
certify_hospital(tanyas_client, project_identifier, hampton_hospital_guid)


In [ ]:

display_engine_activity_c(row_limit=2, width=170)


----

## Creating the pipelines for each hospital

The clinical trial is about to start, and so Peter creates the pipelines for the three hospitals, not realizing that Old Market Hospital is not ready.

-----

In [ ]:
peters_client = EgeriaTech(peters_view_server, peters_view_server_url, peters_user_id, peters_user_pwd)
token = peters_client.create_egeria_bearer_token()

In [ ]:
oak_dene_landing_area_directory_name=onboard_hospital(peters_client, project_identifier, project_directory_name, oak_dene_hospital_guid, oak_dene_landing_folder_directory)
old_market_landing_area_directory_name=onboard_hospital(peters_client, project_identifier, project_directory_name, old_market_hospital_guid, old_market_landing_folder_directory)
hampton_landing_area_directory_name=onboard_hospital(peters_client, project_identifier, project_directory_name, hampton_hospital_guid, hampton_landing_folder_directory)

----

Notice that the process for Old Market Hospital fails because it is not certified.

----

In [ ]:

display_engine_activity_c(row_limit=3, width=170)


----

Notice that the pipelines for Oak Dene and Hampton Hospital are set up ... but nothing for Old Market.  This means that data from uncertified hospitals cannot be accidently included in the clinical trial.

----

In [ ]:
display_integration_daemon_status(['MaintainLastUpdateDate', 'FilesCataloguer'], paging=True, width=170)

----

And the lineage now includes the data feeds from these hospitals ...

----

In [ ]:

print_information_supply_chains(egeria_client, "Clinical Trial Treatment Validation::" + project_identifier, True, True)


----

### Add data to pipelines

At this point, the processes are set up for Oak Dene Hospital and Hampton Hospital.  It is possible to add the data files to their landing area and they will be systematically added to the data lake folder.   No files are processed for Old Market Hospital.

----

The data onboarding pipeline is called `Onboard Landing Area Files For Teddy Bear Drop Foot Project`.

----

In [ ]:

onboarding_pipeline_guid = peters_client.get_guid_for_name(onboarding_pipeline_name) 

process_graph = peters_client.get_gov_action_process_graph(onboarding_pipeline_guid)
print_governance_action_process_graph(process_graph)


---

Let's add Oak Dene Hospital's data for week 1 of the clinical trial ...

---

In [ ]:
add_file_to_landing_area(peters_client, oak_dene_source_folder, oak_dene_landing_area_directory_name, "1")

---

The command below prints out the process steps that have run ...

---

In [ ]:
oak_dene_onboarding_process_name="Coco::GovernanceActionProcess::" + project_identifier + "::" + oak_dene_hospital_name + "::WeeklyMeasurements::Onboarding"
print(oak_dene_onboarding_process_name)
print_process_instances(peters_client, oak_dene_onboarding_process_name)

In [ ]:
add_file_to_landing_area(peters_client, hampton_source_folder, hampton_landing_area_directory_name, "1")

In [ ]:
hampton_onboarding_process_name="Coco::GovernanceActionProcess::" + project_identifier + "::" + hampton_hospital_name + "::WeeklyMeasurements::Onboarding"
print(hampton_onboarding_process_name)
print_process_instances(peters_client, hampton_onboarding_process_name)

In [ ]:
add_file_to_landing_area(peters_client, oak_dene_source_folder, oak_dene_landing_area_directory_name, "2")
add_file_to_landing_area(peters_client, hampton_source_folder, hampton_landing_area_directory_name, "2")

In [ ]:
print_process_instances(peters_client, oak_dene_onboarding_process_name)
print_process_instances(peters_client, hampton_onboarding_process_name)

In [ ]:
display_engine_activity_c(row_limit=4, width=170)

In [ ]:
print_information_supply_chains(peters_client, "Clinical Trial Treatment Validation::" + project_identifier, True, False)


In [ ]:
print_information_supply_chains(peters_client, "Clinical Trial Treatment Validation::" + project_identifier, True, True)


----
## Callie looks for status



In [ ]:

callies_client = EgeriaTech(callies_view_server, callies_view_server_url, callies_user_id, callies_user_pwd)
token = callies_client.create_egeria_bearer_token()

print_solution_roles(callies_client, "New Treatment Data Scientist")


In [ ]:
postgres_db_schema_asset_name = "PostgreSQL Relational Database Schema::LocalPostgreSQL1::clinical_trials.teddy_bear_drop_foot"

print_asset_graph(callies_client, postgres_db_schema_asset_name)

In [ ]:
oak_dene_landing_area_folder = "Data Folder::.landing-area/hospitals/oak-dene/clinical-trials/teddy-bear-drop-foot"

print_asset_graph(callies_client, oak_dene_landing_area_folder)
print_asset_isc_graph(callies_client, oak_dene_landing_area_folder)
print_asset_field_level_lineage_graph(callies_client, oak_dene_landing_area_folder)
print_asset_action_graph(callies_client, oak_dene_landing_area_folder)
print_asset_local_lineage_graph(callies_client, oak_dene_landing_area_folder)
print_files_in_folder(callies_client, oak_dene_landing_area_folder, False)
print_files_in_folder(callies_client, oak_dene_landing_area_folder, True)

In [ ]:
oak_dene_landing_area_folder = "Data Folder::.landing-area/hospitals/oak-dene/clinical-trials/teddy-bear-drop-foot"

print_asset_graph(callies_client, oak_dene_landing_area_folder)
print_asset_isc_graph(callies_client, oak_dene_landing_area_folder)
print_asset_field_level_lineage_graph(callies_client, oak_dene_landing_area_folder)
print_asset_action_graph(callies_client, oak_dene_landing_area_folder)
print_asset_local_lineage_graph(callies_client, oak_dene_landing_area_folder)
print_files_in_folder(callies_client, oak_dene_landing_area_folder, False)
print_files_in_folder(callies_client, oak_dene_landing_area_folder, True)

In [ ]:
data_lake_folder_unique_name = "DataFolder::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.4-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/teddy-bear-drop-foot/weekly-measurements"

print_asset_graph(callies_client, data_lake_folder_unique_name)
print_asset_isc_graph(callies_client, data_lake_folder_unique_name)
print_asset_field_level_lineage_graph(callies_client, data_lake_folder_unique_name)
print_asset_action_graph(callies_client, data_lake_folder_unique_name)
print_asset_local_lineage_graph(callies_client, data_lake_folder_unique_name)
print_files_in_folder(callies_client, data_lake_folder_unique_name, False)
print_files_in_folder(callies_client, data_lake_folder_unique_name, True)

In [ ]:
integration_connector_unique_name = "Egeria:IntegrationGroup:Default::GeneralFilesMonitorIntegrationConnector"

print_asset_graph(callies_client, integration_connector_unique_name)
print_asset_isc_graph(callies_client, integration_connector_unique_name)
print_asset_field_level_lineage_graph(callies_client, integration_connector_unique_name)
print_asset_action_graph(callies_client, integration_connector_unique_name)
print_asset_local_lineage_graph(callies_client, integration_connector_unique_name)


In [ ]:

print_asset_lineage_graph(callies_client, postgres_db_schema_asset_name)


In [ ]:

print_asset_isc_graph(callies_client, postgres_db_schema_asset_name)


In [ ]:

print_asset_isc_graph(callies_client, "Egeria:IntegrationGroup:Default::GeneralFilesMonitorIntegrationConnector")


In [ ]:

print_asset_field_level_lineage_graph(callies_client, postgres_db_schema_asset_name)


In [ ]:

print_request_for_actions(callies_client, "Failed To Pass Quality Gate")


In [ ]:

data_lake_file_000000 = "DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.4-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/teddy-bear-drop-foot/weekly-measurements/PROJ-CT-TBDF_000000.csv"
data_lake_file_000000 = "DataLake::PROJ-CT-TBDF::WeeklyMeasurements::Oak Dene Hospital::PROJ-CT-TBDF_000000.csv"

print_asset_graph(callies_client, data_lake_file_000000, for_lineage=True)


In [ ]:

data_lake_file_000001 = "DataLake::PROJ-CT-TBDF::WeeklyMeasurements::Hampton Hospital::PROJ-CT-TBDF_000001.csv"

print_asset_graph(callies_client, data_lake_file_000001)
print_asset_local_lineage_graph(callies_client, data_lake_file_000001)
print_asset_lineage_graph(callies_client, data_lake_file_000001)


In [ ]:

data_lake_file_000005 = "DataLake::PROJ-CT-TBDF::WeeklyMeasurements::Old Market Hospital::PROJ-CT-TBDF_000005.csv"

print_asset_graph(callies_client, data_lake_file_000005)
print_asset_local_lineage_graph(callies_client, data_lake_file_000005)
print_asset_lineage_graph(callies_client, data_lake_file_000005)


In [ ]:

print_asset_field_level_lineage_graph(callies_client, data_lake_file_000000)


In [ ]:

print_asset_field_level_lineage_graph(callies_client, data_lake_file_000005)


In [ ]:


guid="8c1a07a8-cd41-4c52-9dff-65ff377ca7e7"
related_list = callies_client.get_all_related_metadata_elements(guid, {"forLineage" : True }, True)
if type(related_list) == str:
    print(related_list)
else:
    mermaid_graph = related_list.get('mermaidGraph')
    if mermaid_graph:
        render_mermaid(mermaid_graph)
                

In [ ]:

print_asset_lineage_graph_by_guid(callies_client, "073e8264-54aa-45be-a16e-e3b9d18e567b")


In [ ]:

print_asset_graph(callies_client, "Unity Catalog Volume::" + uc_server_url + "::clinical_trials.teddy_bear_drop_foot.weekly_measurements")


In [ ]:

body = {
            "class" : "FindRequestBody",
            "metadataElementTypeName": "DataFolder"
}
callies_client.find_metadata_elements(body)

In [ ]:

print_files_in_folder(callies_client, "Data Folder::.landing-area/hospitals/oak-dene/clinical-trials/teddy-bear-drop-foot", True)


In [ ]:

print_asset_lineage_graph(callies_client, "PostgreSQL Relational Database Schema::LocalPostgreSQL1::clinical_trials.teddy_bear_drop_foot")


In [ ]:

print_related_elements_for_name(callies_client, 'CertificationType:: Valid weekly measurements data for ' + project_identifier + 'Clinical Trial', "Certification")

mermaidGraph="---\ntitle: Certifications - Valid data for Teddy Bear Drop Foot Clinical Trial [8a921039-ad5f-454d-ae17-e5a5b69f9333]\n---\nflowchart LR\n%%{init: {\"flowchart\": {\"htmlLabels\": false}} }%%\n\n1@{ shape: doc, label: \"*Certification Type*\n**Valid data for Teddy Bear Drop Foot Clinical Trial**\"}\n2@{ shape: text, label: \"*Title*\n**Valid data for Teddy Bear Drop Foot Clinical Trial**\"}\n3@{ shape: text, label: \"*Summary*\n**The data in this file matches the specification for the Teddy Bear Drop Foot Clinical Trial.**\"}\n2~~~3\n4@{ shape: text, label: \"*Description*\n**Within Project**\"}\n3~~~4\n5@{ shape: tag-doc, label: \"*C S V File*\n**PROJ-CT-TBDF_000003.csv from Hampton Hospital**\"}\n1==>|\"Certification\"|5\n6@{ shape: stadium, label: \"*Survey Action Service*\n**Validate that the data in a weekly measurements file passes data validation checks.**\"}\n5-->|\"certifiedBy\"|6\n7@{ shape: trap-t, label: \"*Person*\n**Peter Profile**\"}\n5-->|\"certifiedBy\"|7\n8@{ shape: tag-doc, label: \"*C S V File*\n**PROJ-CT-TBDF_000002.csv from Oak Dene Hospital**\"}\n1==>|\"Certification\"|8\n8-->|\"certifiedBy\"|6\n8-->|\"certifiedBy\"|7\n9@{ shape: tag-doc, label: \"*C S V File*\n**PROJ-CT-TBDF_000000.csv from Oak Dene Hospital**\"}\n1==>|\"Certification\"|9\n9-->|\"certifiedBy\"|6\n9-->|\"certifiedBy\"|7\nstyle 1 color:#FFFFFF, fill:#006400, stroke:#000000\nstyle 2 color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle 3 color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle 4 color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle 5 color:#000000, fill:#f0e68c, stroke:#004563\nstyle 6 color:#000000, fill:#f0e68c, stroke:#004563\nstyle 7 color:#000000, fill:#FFE599, stroke:#000000\nstyle 8 color:#000000, fill:#f0e68c, stroke:#004563\nstyle 9 color:#000000, fill:#f0e68c, stroke:#004563\n"
render_mermaid(mermaidGraph)


In [ ]:

print_asset_graph(callies_client, "Data File Collection::Validated Incoming Weekly Measurements for the Teddy Bear Drop Foot Clinical Trial.")


In [ ]:

print_asset_lineage_graph(callies_client, "Data File Collection::Validated Incoming Weekly Measurements for the Teddy Bear Drop Foot Clinical Trial.")


In [ ]:

print_asset_lineage_graph(callies_client, "DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.4-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/teddy-bear-drop-foot/weekly-measurements/PROJ-CT-TBDF_000001.csv")


In [ ]:

print_information_supply_chains(callies_client, "Clinical Trial Treatment Validation::" + project_identifier, False)


In [ ]:

print_information_supply_chains(callies_client, "Clinical Trial Treatment Validation::" + project_identifier, True, False)


In [ ]:

print_information_supply_chains(callies_client, "Clinical Trial Treatment Validation::" + project_identifier, True, True)


In [ ]:
print_anchor_graph(callies_client, oak_dene_hospital_name)

In [ ]:

print_solution_components(callies_client, "Pipeline")


In [ ]:

print_solution_components(callies_client, "Treatment Validation Sandbox")


In [ ]:

print_solution_components(callies_client, "Treatment")


In [ ]:

print_solution_components(callies_client, "Populate")


In [ ]:
search_results = callies_client.find_in_asset_domain("Validated Incoming")

print_search_results(search_results)

In [ ]:
search_results = callies_client.find_in_asset_domain("LandingArea::Hampton")

print_search_results(search_results)

In [ ]:
search_results = callies_client.find_in_asset_domain("DataLake::" + project_identifier + "::WeeklyMeasurements::")

print_search_results(search_results)

In [ ]:
search_results = callies_client.find_in_asset_domain(project_identifier + "_000003.csv")

print_search_results(search_results)

In [ ]:
print_asset_graph(callies_client, "Coco:GovernanceActionProcess:ClinicalTrials:WeeklyMeasurements:Onboarding")

In [ ]:
print_asset_graph(callies_client, "Apache Airflow DAG:" + airflow_dag_name)

In [ ]:

print_asset_lineage_graph(callies_client, "Apache Airflow DAG:" + airflow_dag_name)


In [ ]:

mermaidGraph="---\ntitle: Certifications - Valid data for Teddy Bear Drop Foot Clinical Trial [8a921039-ad5f-454d-ae17-e5a5b69f9333]\n---\nflowchart LR\n%%{init: {\"flowchart\": {\"htmlLabels\": false}} }%%\n\n1@{ shape: doc, label: \"*Certification Type*\n**Valid data for Teddy Bear Drop Foot Clinical Trial**\"}\n2@{ shape: text, label: \"*Title*\n**Valid data for Teddy Bear Drop Foot Clinical Trial**\"}\n3@{ shape: text, label: \"*Summary*\n**The data in this file matches the specification for the Teddy Bear Drop Foot Clinical Trial.**\"}\n2~~~3\n4@{ shape: text, label: \"*Description*\n**Within Project**\"}\n3~~~4\n5@{ shape: tag-doc, label: \"*C S V File*\n**Oak Dene Hospital teddy bear measurements received on Wed May 28 18:08:30 BST 2025 for Teddy Bear Drop Foot Clinical Trial**\"}\n1==>|\"Certification\"|5\n6@{ shape: tag-doc, label: \"*C S V File*\n**Oak Dene Hospital teddy bear measurements received on Wed May 28 18:08:30 BST 2025 for Teddy Bear Drop Foot Clinical Trial**\"}\n1==>|\"Certification\"|6\n7@{ shape: tag-doc, label: \"*C S V File*\n**Oak Dene Hospital teddy bear measurements received on Wed May 28 18:08:30 BST 2025 for Teddy Bear Drop Foot Clinical Trial**\"}\n1==>|\"Certification\"|7\nstyle 1 color:#FFFFFF, fill:#006400, stroke:#000000\nstyle 2 color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle 3 color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle 4 color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle 5 color:#000000, fill:#f0e68c, stroke:#004563\nstyle 6 color:#000000, fill:#f0e68c, stroke:#004563\nstyle 7 color:#000000, fill:#f0e68c, stroke:#004563\n"
print(mermaidGraph)
render_mermaid(mermaidGraph)


----
### Certifying Old Market Hospital

At last, Old Market Hospital is readdy to join the clinical trial.  Tanya certifies the hospital.

---

In [ ]:

tanyas_client = EgeriaTech(tanyas_view_server, tanyas_view_server_url, tanyas_user_id, tanyas_user_pwd)
token = tanyas_client.create_egeria_bearer_token()

certify_hospital(tanyas_client, project_identifier, old_market_hospital_guid)


In [ ]:

display_engine_activity_c(row_limit=1,width=170)


----

Peter is then able to create the onboarding pipeline for Old Market Hospital

----

In [ ]:

peters_client = EgeriaTech(peters_view_server, peters_view_server_url, peters_user_id, peters_user_pwd)
token = peters_client.create_egeria_bearer_token()

old_market_landing_area_directory_name=onboard_hospital(peters_client, project_identifier, project_directory_name, old_market_hospital_guid, old_market_landing_folder_directory)


In [ ]:

display_engine_activity_c(row_limit=1,width=170)


In [ ]:

display_integration_daemon_status(['MaintainLastUpdateDate', 'FilesCataloguer'], paging=True, width=170)


----

Now any files that are added to Old Market Hospital's landing area are ingested into the data lake. Add week 1 data.

----

In [ ]:
add_file_to_landing_area(peters_client, old_market_source_folder, old_market_landing_area_directory_name, "1")

----

Check that the pipeline is running ok

----

In [ ]:
display_engine_activity_c(row_limit=1, width=170)

----

Try week 2

-----

In [ ]:
add_file_to_landing_area(peters_client, old_market_source_folder, old_market_landing_area_directory_name, "2")

In [ ]:
display_engine_activity_c(row_limit=3, width=170)

-----

Now Old Market Hospital is caught up and the subsequent weeks can continue in synchronization.

----

In [ ]:

add_file_to_landing_area(peters_client, oak_dene_source_folder, oak_dene_landing_area_directory_name, "3")
add_file_to_landing_area(peters_client, hampton_source_folder, hampton_landing_area_directory_name, "3")
add_file_to_landing_area(peters_client, old_market_source_folder, old_market_landing_area_directory_name, "3")


In [ ]:
display_engine_activity_c(row_limit=3, width=170)

In [ ]:

add_file_to_landing_area(peters_client, oak_dene_source_folder, oak_dene_landing_area_directory_name, "4")
add_file_to_landing_area(peters_client, hampton_source_folder, hampton_landing_area_directory_name, "4")
add_file_to_landing_area(peters_client, old_market_source_folder, old_market_landing_area_directory_name, "4")


In [ ]:
display_engine_activity_c(row_limit=3, width=170)

In [ ]:

add_file_to_landing_area(peters_client, oak_dene_source_folder, oak_dene_landing_area_directory_name, "5")
add_file_to_landing_area(peters_client, hampton_source_folder, hampton_landing_area_directory_name, "5")
add_file_to_landing_area(peters_client, old_market_source_folder, old_market_landing_area_directory_name, "5")


In [ ]:
display_engine_activity_c(row_limit=3, width=170)

----
### After all hospitals are registered ...

---

In [ ]:

callies_client = EgeriaTech(callies_view_server, callies_view_server_url, callies_user_id, callies_user_pwd)
token = callies_client.create_egeria_bearer_token()


In [ ]:

print_asset_lineage_graph(callies_client, "PostgreSQL Relational Database Schema::LocalPostgreSQL1::clinical_trials.teddy_bear_drop_foot")


In [ ]:

print_information_supply_chains(callies_client, "Clinical Trial Treatment Validation::PROJ-CT-TBDF", add_implementation=True, for_lineage=True)


------

## Dragon Breath Clinical Trial

The cells below step through setting up a new clinical trial using the same steps used for Teddy Bear Dop Foot to show the use of both the standard and specialized resources for a clinical trial.


----

In [ ]:

print()

project_identifier="PROJ-CT-DB"
project_name="Dragon Breath Clinical Trial"
project_description="Clinical trial related to the new treatment to enhance the volume and quality of the lungs."
project_directory_name="dragon-breath"
project_schema_name="dragon_breath"

print("Project " + project_identifier + ": " + project_name)
print("  " + project_description)


In [ ]:

tessas_client = EgeriaTech(tessas_view_server, tessas_view_server_url, tessas_user_id, tessas_user_pwd)
token = tessas_client.create_egeria_bearer_token()

set_up_clinical_trial(tessas_client,
                      parent_project_guid,
                      clinical_trial_owner_guid,
                      clinical_trial_manager_guid,
                      project_manager_guid,
                      data_engineer_guid,
                      integration_developer_guid,
                      data_scientist_guid,
                      project_identifier, 
                      project_name,
                      project_description) 


In [ ]:

display_engine_activity_c(row_limit=1,width=170)


In [ ]:

print_matching_gov_action_processes(tessas_client, project_identifier)


project_guid=tessas_client.get_element_guid_by_unique_name("Project::" + project_identifier + "::" + project_name)
print_project_graph(tessas_client, project_guid)


In [ ]:

tanyas_client = EgeriaTech(tanyas_view_server, tanyas_view_server_url, tanyas_user_id, tanyas_user_pwd)
token = tanyas_client.create_egeria_bearer_token()

nominate_hospital(tanyas_client, project_identifier, bowden_arrow_hospital_guid, bowden_arrow_contact_person_guid)


In [ ]:

display_engine_activity_c(row_limit=1,width=170)


In [ ]:

peters_client = EgeriaTech(peters_view_server, peters_view_server_url, peters_user_id, peters_user_pwd)
token = peters_client.create_egeria_bearer_token()

set_up_data_lake(peters_client, project_identifier, project_name, project_directory_name, project_schema_name)


In [ ]:

display_engine_activity_c(row_limit=1,width=170)


In [ ]:

tanyas_client = EgeriaTech(tanyas_view_server, tanyas_view_server_url, tanyas_user_id, tanyas_user_pwd)
token = tanyas_client.create_egeria_bearer_token()

certify_hospital(tanyas_client, project_identifier, bowden_arrow_hospital_guid)


In [ ]:

display_engine_activity_c(row_limit=1,width=170)


In [ ]:
peters_client = EgeriaTech(peters_view_server, peters_view_server_url, peters_user_id, peters_user_pwd)
token = peters_client.create_egeria_bearer_token()

bowden_arrow_landing_area_directory_name=onboard_hospital(peters_client, project_identifier, project_directory_name, bowden_arrow_hospital_guid, bowden_arrow_landing_folder_directory)

In [ ]:

display_engine_activity_c(row_limit=2,width=170)


In [ ]:
print_information_supply_chains(peters_client, "Clinical Trial Treatment Validation::" + project_identifier, True, True)

In [ ]:

print_asset_lineage_graph(peters_client, "Data File Collection::Validated Incoming Weekly Measurements for the " + project_name + ".")


In [ ]:

integration_connector_unique_name = "Egeria:IntegrationGroup:Default::GeneralFilesMonitorIntegrationConnector"

print_asset_lineage_graph(peters_client, integration_connector_unique_name)


-----

## Falcon Feather Mite


In [ ]:

print()

project_identifier="PROJ-CT-FFM"
project_name="Falcon Feather Mite Clinical Trial"
project_description="Clinical trial related to the new treatment to cuew feather mites in falcons."
project_directory_name="falcon-feather-mite"
project_schema_name="falcon-feather-mite"

print("Project " + project_identifier + ": " + project_name)
print("  " + project_description)


In [ ]:

tessas_client = EgeriaTech(tessas_view_server, tessas_view_server_url, tessas_user_id, tessas_user_pwd)
token = tessas_client.create_egeria_bearer_token()

set_up_clinical_trial(tessas_client,
                      parent_project_guid,
                      clinical_trial_owner_guid,
                      clinical_trial_manager_guid,
                      project_manager_guid,
                      data_engineer_guid,
                      integration_developer_guid,
                      data_scientist_guid,
                      project_identifier, 
                      project_name,
                      project_description) 


In [ ]:

display_engine_activity_c(row_limit=1,width=170)


In [ ]:

print_matching_gov_action_processes(tessas_client, project_identifier)


project_guid=tessas_client.get_element_guid_by_unique_name("Project::" + project_identifier + "::" + project_name)
print_project_graph(tessas_client, project_guid)


In [ ]:

tanyas_client = EgeriaTech(tanyas_view_server, tanyas_view_server_url, tanyas_user_id, tanyas_user_pwd)
token = tanyas_client.create_egeria_bearer_token()

nominate_hospital(tanyas_client, project_identifier, old_market_hospital_guid, old_market_contact_person_guid)
nominate_hospital(tanyas_client, project_identifier, hampton_hospital_guid, hampton_contact_person_guid)


In [ ]:

display_engine_activity_c(row_limit=1,width=170)


In [ ]:

peters_client = EgeriaTech(peters_view_server, peters_view_server_url, peters_user_id, peters_user_pwd)
token = peters_client.create_egeria_bearer_token()

set_up_data_lake(peters_client, project_identifier, project_name, project_directory_name, project_schema_name)


In [ ]:

display_engine_activity_c(row_limit=1,width=170)


In [ ]:

tanyas_client = EgeriaTech(tanyas_view_server, tanyas_view_server_url, tanyas_user_id, tanyas_user_pwd)
token = tanyas_client.create_egeria_bearer_token()

certify_hospital(tanyas_client, project_identifier, old_market_hospital_guid)
certify_hospital(tanyas_client, project_identifier, hampton_hospital_guid)


In [ ]:

display_engine_activity_c(row_limit=1,width=170)


In [ ]:
peters_client = EgeriaTech(peters_view_server, peters_view_server_url, peters_user_id, peters_user_pwd)
token = peters_client.create_egeria_bearer_token()

old_market_landing_area_directory_name=onboard_hospital(peters_client, project_identifier, project_directory_name, old_market_hospital_guid, old_market_landing_folder_directory)
hampton_landing_area_directory_name=onboard_hospital(peters_client, project_identifier, project_directory_name, hampton_hospital_guid, hampton_landing_folder_directory)

In [ ]:

display_engine_activity_c(row_limit=1,width=170)


In [ ]:
print_information_supply_chains(peters_client, "Clinical Trial Treatment Validation::" + project_identifier, True, True)

In [ ]:

print_asset_lineage_graph(peters_client, "Data File Collection::Validated Incoming Weekly Measurements for the " + project_name + ".")


-----

## Werewolf Transformation


In [ ]:

print()

project_identifier="PROJ-CT-WTF"
project_name="Werewolf Transformation Clinical Trial"
project_description="Clinical trial related to the new treatment to minimise or prevent werewolf transformation."
project_directory_name="werewolf-transformation"
project_schema_name="werewolf_transformation"

print("Project " + project_identifier + ": " + project_name)
print("  " + project_description)


In [ ]:

tessas_client = EgeriaTech(tessas_view_server, tessas_view_server_url, tessas_user_id, tessas_user_pwd)
token = tessas_client.create_egeria_bearer_token()

set_up_clinical_trial(tessas_client,
                      parent_project_guid,
                      clinical_trial_owner_guid,
                      clinical_trial_manager_guid,
                      project_manager_guid,
                      data_engineer_guid,
                      integration_developer_guid,
                      data_scientist_guid,
                      project_identifier, 
                      project_name,
                      project_description) 


In [ ]:

display_engine_activity_c(row_limit=1,width=170)


In [ ]:

print_matching_gov_action_processes(tessas_client, project_identifier)


project_guid=tessas_client.get_element_guid_by_unique_name("Project::" + project_identifier + "::" + project_name)
print_project_graph(tessas_client, project_guid)


In [ ]:

tanyas_client = EgeriaTech(tanyas_view_server, tanyas_view_server_url, tanyas_user_id, tanyas_user_pwd)
token = tanyas_client.create_egeria_bearer_token()

nominate_hospital(tanyas_client, project_identifier, oak_dene_hospital_guid, oak_dene_contact_person_guid)
nominate_hospital(tanyas_client, project_identifier, old_market_hospital_guid, old_market_contact_person_guid)
nominate_hospital(tanyas_client, project_identifier, hampton_hospital_guid, hampton_contact_person_guid)
nominate_hospital(tanyas_client, project_identifier, bowden_arrow_hospital_guid, bowden_arrow_contact_person_guid)


In [ ]:

display_engine_activity_c(row_limit=4, width=150)


In [ ]:

peters_client = EgeriaTech(peters_view_server, peters_view_server_url, peters_user_id, peters_user_pwd)
token = peters_client.create_egeria_bearer_token()

set_up_data_lake(peters_client, project_identifier, project_name, project_directory_name, project_schema_name)


In [ ]:

display_engine_activity_c(row_limit=1, width=150)


In [ ]:

tanyas_client = EgeriaTech(tanyas_view_server, tanyas_view_server_url, tanyas_user_id, tanyas_user_pwd)
token = tanyas_client.create_egeria_bearer_token()

certify_hospital(tanyas_client, project_identifier, oak_dene_hospital_guid)
certify_hospital(tanyas_client, project_identifier, old_market_hospital_guid)
certify_hospital(tanyas_client, project_identifier, hampton_hospital_guid)
certify_hospital(tanyas_client, project_identifier, bowden_arrow_hospital_guid)


In [ ]:

display_engine_activity_c(row_limit=4, width=150)


In [ ]:
peters_client = EgeriaTech(peters_view_server, peters_view_server_url, peters_user_id, peters_user_pwd)
token = peters_client.create_egeria_bearer_token()

oak_dene_landing_area_directory_name=onboard_hospital(peters_client, project_identifier, project_directory_name, oak_dene_hospital_guid, oak_dene_landing_folder_directory)
old_market_landing_area_directory_name=onboard_hospital(peters_client, project_identifier, project_directory_name, old_market_hospital_guid, old_market_landing_folder_directory)
hampton_landing_area_directory_name=onboard_hospital(peters_client, project_identifier, project_directory_name, hampton_hospital_guid, hampton_landing_folder_directory)
bowden_arrow_landing_area_directory_name=onboard_hospital(peters_client, project_identifier, project_directory_name, bowden_arrow_hospital_guid, bowden_arrow_landing_folder_directory)

In [ ]:

print_information_supply_chains(peters_client, "Clinical Trial Treatment Validation::" + project_identifier, True, True)


In [ ]:

print_information_supply_chains(peters_client, "Clinical Trial Treatment Validation::" + project_identifier, True, True)


In [ ]:

add_file_to_landing_area(peters_client, oak_dene_source_folder, oak_dene_landing_area_directory_name, "1")


In [ ]:

print_information_supply_chains(peters_client, "Clinical Trial Treatment Validation::" + project_identifier, True, True)


In [ ]:

print_asset_lineage_graph(peters_client, "Data File Collection::Validated Incoming Weekly Measurements for the " + project_name + ".")


In [ ]:
integration_connector_unique_name = "Egeria:IntegrationGroup:Default::GeneralFilesMonitorIntegrationConnector"

print_asset_graph(peters_client, integration_connector_unique_name)
print_asset_isc_graph(peters_client, integration_connector_unique_name)
print_asset_field_level_lineage_graph(peters_client, integration_connector_unique_name)
print_asset_action_graph(peters_client, integration_connector_unique_name)
print_asset_local_lineage_graph(peters_client, integration_connector_unique_name)
print_asset_lineage_graph(peters_client, integration_connector_unique_name)


In [ ]:
oak_dene_landing_area_folder = "Data Folder::.landing-area/hospitals/oak-dene/clinical-trials/teddy-bear-drop-foot"

print_files_in_folder(peters_client, oak_dene_landing_area_folder)

print_asset_graph(peters_client, oak_dene_landing_area_folder)
print_asset_isc_graph(peters_client, oak_dene_landing_area_folder)


In [ ]:
data_lake_file_000000 = "DataLake::PROJ-CT-TBDF::WeeklyMeasurements::Oak Dene Hospital::PROJ-CT-TBDF_000000.csv"

print_asset_lineage_graph(peters_client, data_lake_file_000000)
print_asset_lineage_graph(peters_client, data_lake_file_000000, relationship_types=["DataFlow", "DataMapping"])


In [ ]:

postgres_db_schema_asset_name = "PostgreSQL Relational Database Schema::LocalPostgreSQL1::clinical_trials.teddy_bear_drop_foot"

print_asset_graph(callies_client, postgres_db_schema_asset_name)
print_asset_lineage_graph(callies_client, postgres_db_schema_asset_name)


In [ ]:

print_files_in_folder(peters_client, oak_dene_landing_area_folder)

print_files_in_folder(peters_client, oak_dene_landing_area_folder, for_lineage=True)


In [ ]:

mermaidGraph="---\ntitle: Lineage Analysis Graph for Asset - GeneralFilesMonitorIntegrationConnector [1b98cdac-dd0a-4621-93db-99ef5a1098bc]\n---\nflowchart TD\n%%{init: {\"flowchart\": {\"htmlLabels\": false}} }%%\n\n1@{ shape: hex, label: \"*Integration Connector*\n**GeneralFilesMonitorIntegrationConnector**\"}\n2@{ shape: lin-cyl, label: \"*Data Folder*\n**proj-ct-tbdf**\"}\n3@{ shape: tri, label: \"*Organization*\n**Old Market Hospital**\"}\n4@{ shape: lin-cyl, label: \"*Data Folder*\n**proj-ct-tbdf**\"}\n5@{ shape: tri, label: \"*Organization*\n**Hampton Hospital**\"}\n6@{ shape: lin-cyl, label: \"*Data Folder*\n**proj-ct-tbdf**\"}\n7@{ shape: tri, label: \"*Organization*\n**Oak Dene Hospital**\"}\n8@{ shape: tag-rect, label: \"*Governance Action Process*\n**Onboard Hampton Hospital Landing Area Files for Teddy Bear Drop Foot Clinical Trial**\"}\n9@{ shape: cyl, label: \"*Data File Collection*\n**Validated Incoming Weekly Measurements for the Teddy Bear Drop Foot Clinical Trial.**\"}\n10@{ shape: tag-doc, label: \"*C S V File*\n**PROJ-CT-TBDF_000000.csv from Oak Dene Hospital**\"}\n11@{ shape: tag-doc, label: \"*C S V File*\n**PROJ-CT-TBDF_000003.csv from Hampton Hospital**\"}\n12@{ shape: lin-cyl, label: \"*Data Folder*\n**weekly_measurements**\"}\n13@{ shape: tag-doc, label: \"*C S V File*\n**PROJ-CT-TBDF_000002.csv from Old Market Hospital**\"}\n14@{ shape: subproc, label: \"*Deployed Software Component*\n**populate_teddy_bear_drop_foot_sandbox**\"}\n15@{ shape: win-pane, label: \"*Deployed Database Schema*\n**clinical_trials.teddy_bear_drop_foot**\"}\n16@{ shape: tag-rect, label: \"*Governance Action Process*\n**Onboard Oak Dene Hospital Landing Area Files for Teddy Bear Drop Foot Clinical Trial**\"}\n17@{ shape: tag-rect, label: \"*Governance Action Process*\n**Onboard Old Market Hospital Landing Area Files for Teddy Bear Drop Foot Clinical Trial**\"}\n4==>|\"new file notification [DataFlow]\"|1\n2==>|\"new file notification [DataFlow]\"|1\n6==>|\"new file notification [DataFlow]\"|1\n9==>|\"DataSetContent\"|10\n16==>|\"delivered validated file [DataFlow]\"|9\n1==>|\"new file trigger [DataFlow]\"|16\n9==>|\"get files from storage [ProcessCall]\"|12\n9==>|\"Copy Certified Data [DataFlow]\"|14\n9==>|\"DataSetContent\"|13\n3==>|\"publish weekly measurements [DataFlow]\"|2\n7==>|\"publish weekly measurements [DataFlow]\"|6\n14==>|\"Save Certified Data [DataFlow]\"|15\n9==>|\"DataSetContent\"|11\n1==>|\"new file trigger [DataFlow]\"|8\n1==>|\"new file trigger [DataFlow]\"|17\n5==>|\"publish weekly measurements [DataFlow]\"|4\n8==>|\"delivered validated file [DataFlow]\"|9\n17==>|\"delivered validated file [DataFlow]\"|9\nstyle 11 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 12 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 13 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 14 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 15 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 16 color:#000000, fill:#53bbb4, stroke:#000000\nstyle 17 color:#000000, fill:#53bbb4, stroke:#000000\nstyle 1 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 2 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 3 color:#000000, fill:#FFE599, stroke:#000000\nstyle 4 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 5 color:#000000, fill:#FFE599, stroke:#000000\nstyle 6 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 7 color:#000000, fill:#FFE599, stroke:#000000\nstyle 8 color:#000000, fill:#53bbb4, stroke:#000000\nstyle 9 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 10 color:#000000, fill:#bdb76b, stroke:#004563\n"

print(mermaidGraph)
render_mermaid(mermaidGraph)

edgeMermaidGraph="---\ntitle: Lineage Edge Analysis Graph for Asset - GeneralFilesMonitorIntegrationConnector [1b98cdac-dd0a-4621-93db-99ef5a1098bc]\n---\nflowchart LR\n%%{init: {\"flowchart\": {\"htmlLabels\": false}} }%%\n\n1@{ shape: stadium, label: \"*Integration Connector*\n**GeneralFilesMonitorIntegrationConnector**\"}\n2@{ shape: tri, label: \"*Organization*\n**Old Market Hospital**\"}\n2==>|\"UltimateSource\"|1\n3@{ shape: tri, label: \"*Organization*\n**Hampton Hospital**\"}\n3==>|\"UltimateSource\"|1\n4@{ shape: tri, label: \"*Organization*\n**Oak Dene Hospital**\"}\n4==>|\"UltimateSource\"|1\n5@{ shape: tag-doc, label: \"*C S V File*\n**PROJ-CT-TBDF_000000.csv from Oak Dene Hospital**\"}\n1==>|\"UltimateDestination\"|5\n6@{ shape: tag-doc, label: \"*C S V File*\n**PROJ-CT-TBDF_000003.csv from Hampton Hospital**\"}\n1==>|\"UltimateDestination\"|6\n7@{ shape: lin-cyl, label: \"*Data Folder*\n**weekly_measurements**\"}\n1==>|\"UltimateDestination\"|7\n8@{ shape: tag-doc, label: \"*C S V File*\n**PROJ-CT-TBDF_000002.csv from Old Market Hospital**\"}\n1==>|\"UltimateDestination\"|8\n9@{ shape: win-pane, label: \"*Deployed Database Schema*\n**clinical_trials.teddy_bear_drop_foot**\"}\n1==>|\"UltimateDestination\"|9\nstyle 1 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 2 color:#000000, fill:#FFE599, stroke:#000000\nstyle 3 color:#000000, fill:#FFE599, stroke:#000000\nstyle 4 color:#000000, fill:#FFE599, stroke:#000000\nstyle 5 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 6 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 7 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 8 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 9 color:#000000, fill:#bdb76b, stroke:#004563\n"

print(edgeMermaidGraph)
render_mermaid(edgeMermaidGraph)


In [ ]:

mermaidGraph="---\ntitle: Lineage Analysis Graph for Asset - GeneralFilesMonitorIntegrationConnector [1b98cdac-dd0a-4621-93db-99ef5a1098bc]\n---\nflowchart TD\n%%{init: {\"flowchart\": {\"htmlLabels\": false}} }%%\n\n1@{ shape: hex, label: \"*Integration Connector*\n**GeneralFilesMonitorIntegrationConnector**\"}\n2@{ shape: lin-cyl, label: \"*Data Folder*\n**proj-ct-wtf**\"}\n3@{ shape: tri, label: \"*Organization*\n**Oak Dene Hospital**\"}\n4@{ shape: lin-cyl, label: \"*Data Folder*\n**proj-ct-ffm**\"}\n5@{ shape: tri, label: \"*Organization*\n**Hampton Hospital**\"}\n6@{ shape: lin-cyl, label: \"*Data Folder*\n**proj-ct-ffm**\"}\n7@{ shape: tri, label: \"*Organization*\n**Old Market Hospital**\"}\n8@{ shape: lin-cyl, label: \"*Data Folder*\n**proj-ct-db**\"}\n9@{ shape: tri, label: \"*Organization*\n**Bowden Arrow Hospital**\"}\n10@{ shape: lin-cyl, label: \"*Data Folder*\n**proj-ct-tbdf**\"}\n11@{ shape: lin-cyl, label: \"*Data Folder*\n**proj-ct-tbdf**\"}\n12@{ shape: lin-cyl, label: \"*Data Folder*\n**proj-ct-wtf**\"}\n13@{ shape: lin-cyl, label: \"*Data Folder*\n**proj-ct-wtf**\"}\n14@{ shape: lin-cyl, label: \"*Data Folder*\n**proj-ct-wtf**\"}\n15@{ shape: lin-cyl, label: \"*Data Folder*\n**proj-ct-tbdf**\"}\n16@{ shape: tag-rect, label: \"*Governance Action Process*\n**Onboard Hampton Hospital Landing Area Files for Teddy Bear Drop Foot Clinical Trial**\"}\n17@{ shape: cyl, label: \"*Data File Collection*\n**Validated Incoming Weekly Measurements for the Teddy Bear Drop Foot Clinical Trial.**\"}\n18@{ shape: tag-doc, label: \"*C S V File*\n**PROJ-CT-TBDF_000000.csv from Oak Dene Hospital**\"}\n19@{ shape: tag-doc, label: \"*C S V File*\n**PROJ-CT-TBDF_000003.csv from Hampton Hospital**\"}\n20@{ shape: lin-cyl, label: \"*Data Folder*\n**weekly_measurements**\"}\n21@{ shape: tag-doc, label: \"*C S V File*\n**PROJ-CT-TBDF_000002.csv from Old Market Hospital**\"}\n22@{ shape: subproc, label: \"*Deployed Software Component*\n**populate_teddy_bear_drop_foot_sandbox**\"}\n23@{ shape: win-pane, label: \"*Deployed Database Schema*\n**clinical_trials.teddy_bear_drop_foot**\"}\n24@{ shape: tag-rect, label: \"*Governance Action Process*\n**Onboard Oak Dene Hospital Landing Area Files for Werewolf Transformation Clinical Trial**\"}\n25@{ shape: cyl, label: \"*Data File Collection*\n**Validated Incoming Weekly Measurements for the Werewolf Transformation Clinical Trial.**\"}\n26@{ shape: lin-cyl, label: \"*Data Folder*\n**weekly_measurements**\"}\n27@{ shape: subproc, label: \"*Deployed Software Component*\n**populate_werewolf_transformation_sandbox**\"}\n28@{ shape: win-pane, label: \"*Deployed Database Schema*\n**clinical_trials.werewolf_transformation**\"}\n29@{ shape: tag-rect, label: \"*Governance Action Process*\n**Onboard Hampton Hospital Landing Area Files for Falcon Feather Mite Clinical Trial**\"}\n30@{ shape: cyl, label: \"*Data File Collection*\n**Validated Incoming Weekly Measurements for the Falcon Feather Mite Clinical Trial.**\"}\n31@{ shape: lin-cyl, label: \"*Data Folder*\n**weekly_measurements**\"}\n32@{ shape: subproc, label: \"*Deployed Software Component*\n**populate_falcon-feather-mite_sandbox**\"}\n33@{ shape: win-pane, label: \"*Deployed Database Schema*\n**clinical_trials.falcon-feather-mite**\"}\n34@{ shape: tag-rect, label: \"*Governance Action Process*\n**Onboard Bowden Arrow Hospital Landing Area Files for Dragon Breath Clinical Trial**\"}\n35@{ shape: cyl, label: \"*Data File Collection*\n**Validated Incoming Weekly Measurements for the Dragon Breath Clinical Trial.**\"}\n36@{ shape: subproc, label: \"*Deployed Software Component*\n**populate_dragon_breath_sandbox**\"}\n37@{ shape: win-pane, label: \"*Deployed Database Schema*\n**clinical_trials.dragon_breath**\"}\n38@{ shape: lin-cyl, label: \"*Data Folder*\n**weekly_measurements**\"}\n39@{ shape: tag-rect, label: \"*Governance Action Process*\n**Onboard Hampton Hospital Landing Area Files for Werewolf Transformation Clinical Trial**\"}\n40@{ shape: tag-rect, label: \"*Governance Action Process*\n**Onboard Oak Dene Hospital Landing Area Files for Teddy Bear Drop Foot Clinical Trial**\"}\n41@{ shape: tag-rect, label: \"*Governance Action Process*\n**Onboard Bowden Arrow Hospital Landing Area Files for Werewolf Transformation Clinical Trial**\"}\n42@{ shape: tag-rect, label: \"*Governance Action Process*\n**Onboard Old Market Hospital Landing Area Files for Falcon Feather Mite Clinical Trial**\"}\n43@{ shape: tag-rect, label: \"*Governance Action Process*\n**Onboard Old Market Hospital Landing Area Files for Werewolf Transformation Clinical Trial**\"}\n44@{ shape: tag-rect, label: \"*Governance Action Process*\n**Onboard Old Market Hospital Landing Area Files for Teddy Bear Drop Foot Clinical Trial**\"}\n10==>|\"new file notification [DataFlow]\"|1\n1-->|\"new file trigger [DataFlow]\"|24\n35-->|\"get files from storage [ProcessCall]\"|38\n1-->|\"new file trigger [DataFlow]\"|41\n6-->|\"new file notification [DataFlow]\"|1\n1==>|\"new file trigger [DataFlow]\"|16\n1-->|\"new file trigger [DataFlow]\"|29\n34-->|\"delivered validated file [DataFlow]\"|35\n16==>|\"delivered validated file [DataFlow]\"|17\n9-->|\"publish weekly measurements [DataFlow]\"|8\n32-->|\"Save Certified Data [DataFlow]\"|33\n43-->|\"delivered validated file [DataFlow]\"|25\n5-->|\"publish weekly measurements [DataFlow]\"|4\n30-->|\"get files from storage [ProcessCall]\"|31\n7-->|\"publish weekly measurements [DataFlow]\"|6\n17==>|\"DataSetContent\"|18\n36-->|\"Save Certified Data [DataFlow]\"|37\n7-->|\"publish weekly measurements [DataFlow]\"|14\n1==>|\"new file trigger [DataFlow]\"|40\n17==>|\"Copy Certified Data [DataFlow]\"|22\n2-->|\"new file notification [DataFlow]\"|1\n7==>|\"publish weekly measurements [DataFlow]\"|10\n17==>|\"DataSetContent\"|19\n1==>|\"new file trigger [DataFlow]\"|44\n1-->|\"new file trigger [DataFlow]\"|39\n35-->|\"Copy Certified Data [DataFlow]\"|36\n30-->|\"Copy Certified Data [DataFlow]\"|32\n29-->|\"delivered validated file [DataFlow]\"|30\n25-->|\"Copy Certified Data [DataFlow]\"|27\n5-->|\"publish weekly measurements [DataFlow]\"|13\n13-->|\"new file notification [DataFlow]\"|1\n27-->|\"Save Certified Data [DataFlow]\"|28\n1-->|\"new file trigger [DataFlow]\"|43\n1-->|\"new file trigger [DataFlow]\"|34\n22==>|\"Save Certified Data [DataFlow]\"|23\n41-->|\"delivered validated file [DataFlow]\"|25\n9-->|\"publish weekly measurements [DataFlow]\"|12\n8-->|\"new file notification [DataFlow]\"|1\n25-->|\"get files from storage [ProcessCall]\"|26\n11==>|\"new file notification [DataFlow]\"|1\n39-->|\"delivered validated file [DataFlow]\"|25\n15==>|\"new file notification [DataFlow]\"|1\n40==>|\"delivered validated file [DataFlow]\"|17\n3-->|\"publish weekly measurements [DataFlow]\"|2\n14-->|\"new file notification [DataFlow]\"|1\n4-->|\"new file notification [DataFlow]\"|1\n12-->|\"new file notification [DataFlow]\"|1\n17==>|\"get files from storage [ProcessCall]\"|20\n24-->|\"delivered validated file [DataFlow]\"|25\n17==>|\"DataSetContent\"|21\n3==>|\"publish weekly measurements [DataFlow]\"|15\n42-->|\"delivered validated file [DataFlow]\"|30\n5==>|\"publish weekly measurements [DataFlow]\"|11\n1-->|\"new file trigger [DataFlow]\"|42\n44==>|\"delivered validated file [DataFlow]\"|17\nstyle 22 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 44 color:#000000, fill:#53bbb4, stroke:#000000\nstyle 23 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 24 color:#000000, fill:#53bbb4, stroke:#000000\nstyle 25 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 26 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 27 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 28 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 29 color:#000000, fill:#53bbb4, stroke:#000000\nstyle 30 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 31 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 10 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 32 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 11 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 33 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 12 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 34 color:#000000, fill:#53bbb4, stroke:#000000\nstyle 13 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 35 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 14 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 36 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 15 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 37 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 16 color:#000000, fill:#53bbb4, stroke:#000000\nstyle 38 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 17 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 39 color:#000000, fill:#53bbb4, stroke:#000000\nstyle 18 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 19 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 1 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 2 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 3 color:#000000, fill:#FFE599, stroke:#000000\nstyle 4 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 5 color:#000000, fill:#FFE599, stroke:#000000\nstyle 6 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 7 color:#000000, fill:#FFE599, stroke:#000000\nstyle 8 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 9 color:#000000, fill:#FFE599, stroke:#000000\nstyle 40 color:#000000, fill:#53bbb4, stroke:#000000\nstyle 41 color:#000000, fill:#53bbb4, stroke:#000000\nstyle 20 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 42 color:#000000, fill:#53bbb4, stroke:#000000\nstyle 21 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 43 color:#000000, fill:#53bbb4, stroke:#000000\n"

print(mermaidGraph)
render_mermaid(mermaidGraph)

edgeMermaidGraph="---\ntitle: Lineage Edge Analysis Graph for Asset - GeneralFilesMonitorIntegrationConnector [1b98cdac-dd0a-4621-93db-99ef5a1098bc]\n---\nflowchart LR\n%%{init: {\"flowchart\": {\"htmlLabels\": false}} }%%\n\n1@{ shape: stadium, label: \"*Integration Connector*\n**GeneralFilesMonitorIntegrationConnector**\"}\n2@{ shape: tri, label: \"*Organization*\n**Old Market Hospital**\"}\n2==>|\"UltimateSource\"|1\n3@{ shape: tri, label: \"*Organization*\n**Hampton Hospital**\"}\n3==>|\"UltimateSource\"|1\n4@{ shape: tri, label: \"*Organization*\n**Bowden Arrow Hospital**\"}\n4==>|\"UltimateSource\"|1\n5@{ shape: tri, label: \"*Organization*\n**Oak Dene Hospital**\"}\n5==>|\"UltimateSource\"|1\n6@{ shape: tag-doc, label: \"*C S V File*\n**PROJ-CT-TBDF_000000.csv from Oak Dene Hospital**\"}\n1==>|\"UltimateDestination\"|6\n7@{ shape: lin-cyl, label: \"*Data Folder*\n**weekly_measurements**\"}\n1==>|\"UltimateDestination\"|7\n8@{ shape: win-pane, label: \"*Deployed Database Schema*\n**clinical_trials.falcon-feather-mite**\"}\n1==>|\"UltimateDestination\"|8\n9@{ shape: lin-cyl, label: \"*Data Folder*\n**weekly_measurements**\"}\n1==>|\"UltimateDestination\"|9\n10@{ shape: lin-cyl, label: \"*Data Folder*\n**weekly_measurements**\"}\n1==>|\"UltimateDestination\"|10\n11@{ shape: tag-doc, label: \"*C S V File*\n**PROJ-CT-TBDF_000003.csv from Hampton Hospital**\"}\n1==>|\"UltimateDestination\"|11\n12@{ shape: win-pane, label: \"*Deployed Database Schema*\n**clinical_trials.dragon_breath**\"}\n1==>|\"UltimateDestination\"|12\n13@{ shape: lin-cyl, label: \"*Data Folder*\n**weekly_measurements**\"}\n1==>|\"UltimateDestination\"|13\n14@{ shape: tag-doc, label: \"*C S V File*\n**PROJ-CT-TBDF_000002.csv from Old Market Hospital**\"}\n1==>|\"UltimateDestination\"|14\n15@{ shape: win-pane, label: \"*Deployed Database Schema*\n**clinical_trials.teddy_bear_drop_foot**\"}\n1==>|\"UltimateDestination\"|15\n16@{ shape: win-pane, label: \"*Deployed Database Schema*\n**clinical_trials.werewolf_transformation**\"}\n1==>|\"UltimateDestination\"|16\nstyle 11 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 12 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 13 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 14 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 15 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 16 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 1 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 2 color:#000000, fill:#FFE599, stroke:#000000\nstyle 3 color:#000000, fill:#FFE599, stroke:#000000\nstyle 4 color:#000000, fill:#FFE599, stroke:#000000\nstyle 5 color:#000000, fill:#FFE599, stroke:#000000\nstyle 6 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 7 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 8 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 9 color:#000000, fill:#bdb76b, stroke:#004563\nstyle 10 color:#000000, fill:#bdb76b, stroke:#004563\n"

print(edgeMermaidGraph)
render_mermaid(edgeMermaidGraph)


In [16]:

mermaidGraph="---\ntitle: Collection - Oak Dene Hospital's data sharing agreement for clinical trial: PROJ-CT-TBDF [7b310bf7-6fd1-4bca-ae49-8a6b114ada3a]\n---\nflowchart TD\n%%{init: {\"flowchart\": {\"htmlLabels\": false}} }%%\n\n1@{ shape: rect, label: \"*Agreement*\n**Oak Dene Hospital's data sharing agreement for clinical trial: PROJ-CT-TBDF**\"}\n2@{ shape: text, label: \"*Description*\n**This is the basis of the agreement to share data between Oak Dene Hospital and Coco Pharmaceuticals for the purpose of performing the Teddy Bear Drop Foot clinical trial. **\"}\n2~~~1\nsubgraph 3 [Classifications]\ndirection TB\n4@{ shape: rect, label: \"*Anchors*\n**anchorTypeName\n - Agreement\nanchorDomainName\n - Collection\nanchorScopeGUID\n - dce3dd03-28db-4095-b90a-e4231c991589\n**\"}\n5@{ shape: rect, label: \"*Data Sharing Agreement*\n****\"}\nend\nstyle 1 color:#FFFFFF, fill:#6b8e23, stroke:#000000\nstyle 2 color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle 3 color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle 4 color:#FFFFFF, fill:#6b8e23, stroke:#000000\nstyle 5 color:#FFFFFF, fill:#6b8e23, stroke:#000000\n"
render_mermaid(mermaidGraph)



In [17]:

mermaidGraph="---\ntitle: Collection - Data Dictionary for Clinical Trials [f4b067ff-62e7-42f3-a175-bc44158dc910]\n---\nflowchart TD\n%%{init: {\"flowchart\": {\"htmlLabels\": false}} }%%\n\n1@{ shape: rect, label: \"*Collection*\n**Data Dictionary for Clinical Trials**\"}\n2@{ shape: text, label: \"*Description*\n**Holds details of the data fields and classes for clinical trial data.**\"}\n2~~~1\nsubgraph 3 [Classifications]\ndirection TB\n4@{ shape: rect, label: \"*Anchors*\n**anchorTypeName\n - Collection\nanchorDomainName\n - Collection\n**\"}\n5@{ shape: rect, label: \"*Data Dictionary*\n****\"}\nend\nsubgraph 6 [Collection Membership]\ndirection LR\n7@{ shape: rect, label: \"*Collection [ Folder]*\n**Person data definitions**\"}\n8@{ shape: rect, label: \"*Data Field*\n**Patient Identifier**\"}\n7==>|\"Data Field\"|8\n9@{ shape: rect, label: \"*Collection [ Folder]*\n**General data definitions**\"}\n10@{ shape: rect, label: \"*Data Field*\n**Postal Address**\"}\n9==>|\"Data Field\"|10\n11@{ shape: rect, label: \"*Collection [ Folder]*\n**Medical data definitions**\"}\nend\n1-.->6\nstyle 11 color:#000000, fill:#8fbc8f, stroke:#000000\nstyle 1 color:#000000, fill:#d2691e, stroke:#000000\nstyle 2 color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle 3 color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle 4 color:#000000, fill:#d2691e, stroke:#000000\nstyle 5 color:#000000, fill:#d2691e, stroke:#000000\nstyle 6 color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle 7 color:#000000, fill:#8fbc8f, stroke:#000000\nstyle 8 color:#000000, fill:#ffe3cc, stroke:#f9845b\nstyle 9 color:#000000, fill:#8fbc8f, stroke:#000000\nstyle 10 color:#000000, fill:#ffe3cc, stroke:#f9845b\n"
render_mermaid(mermaidGraph)



In [18]:

mermaidGraph="---\ntitle: Data Field - Postal Address [28254160-2daa-4e61-b549-ec8a3d8cfb10]\n---\nflowchart TD\n%%{init: {\"flowchart\": {\"htmlLabels\": false}} }%%\n\n1@{ shape: text, label: \"*Description*\n**Address of a physical location.**\"}\n2@{ shape: rect, label: \"*Data Field*\n**Postal Address**\"}\n3@{ shape: rect, label: \"*Data Field*\n**Country**\"}\n2==>|\"[7] 1..1\"|3\n4@{ shape: rect, label: \"*Data Field*\n**District**\"}\n2==>|\"[3] 0..1\"|4\n5@{ shape: rect, label: \"*Data Field*\n**City**\"}\n2==>|\"[4] 1..1\"|5\n6@{ shape: rect, label: \"*Data Field*\n**Street**\"}\n2==>|\"[2] 0..1\"|6\n7@{ shape: rect, label: \"*Data Field*\n**Postal Code**\"}\n2==>|\"[6] 1..1\"|7\n8@{ shape: rect, label: \"*Data Field*\n**Region**\"}\n2==>|\"[5] 0..1\"|8\n9@{ shape: rect, label: \"*Collection [ Folder]*\n**General data definitions**\"}\n9==>|\"Collection Membership\"|2\nstyle 1 color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle 2 color:#000000, fill:#ffe3cc, stroke:#f9845b\nstyle 3 color:#000000, fill:#ffe3cc, stroke:#f9845b\nstyle 4 color:#000000, fill:#ffe3cc, stroke:#f9845b\nstyle 5 color:#000000, fill:#ffe3cc, stroke:#f9845b\nstyle 6 color:#000000, fill:#ffe3cc, stroke:#f9845b\nstyle 7 color:#000000, fill:#ffe3cc, stroke:#f9845b\nstyle 8 color:#000000, fill:#ffe3cc, stroke:#f9845b\nstyle 9 color:#000000, fill:#8fbc8f, stroke:#000000\n"
render_mermaid(mermaidGraph)

